# Segment 4 Lab 5

Introducing a critical agent - the agent that brings it all together.

# Planning Agent

There are a number of frameworks out there that support building Agentic Workflows.

OpenAI has Swarm, LangChain has LangGraph, Gradio and HuggingFace have offerings, and there's Autogen from Microsoft, Crew.ai and many others.  

Each of these are abstractions on top of APIs to LLMs; some are lightweight, others come with significant functionality.

It's also perfectly possible - and sometimes considerably easier - to build an agentic solution by calling LLMs directly.

There's been considerable convergence on LLM APIs, and it's not clear that there's a need to sign up for one of the agent ecosystems for many use cases.

Anthropic has an [insightful post](https://www.anthropic.com/research/building-effective-agents) on building effective Agentic architectures that's well worth a read.

## Using Tools to give our Agent autonomy

In our case, we're going to create an Agent that uses Tools to make decisions about what to do next.

This is a bit over the top for this simple example, because we know exactly what the Agent is supposed to do. But it allows us to give the Agent some freedom..

Let's see how it works:

In [ ]:
# imports

import os
import json
from dotenv import load_dotenv
from openai import OpenAI

In [ ]:
# Initialization

load_dotenv(override=True)

openai_api_key = os.getenv('OPENAI_API_KEY')
if openai_api_key:
    print(f"OpenAI API Key exists and begins {openai_api_key[:8]}")
else:
    print("OpenAI API Key not set")
    
MODEL = "gpt-4o-mini"
openai = OpenAI()


In [ ]:
# Use the Scanner agent from before

from agents.scanner_agent import ScannerAgent
scanner = ScannerAgent()

# Our tools

The next 3 cells have 3 **fake** functions that we will allow our LLM to call

In [ ]:
def scan_the_internet_for_bargains():
    print(f"Scanning the internet")
    results = scanner.test_scan()
    return results.model_dump()

In [ ]:
def estimate_true_value(description: str) -> str:
    print(f"Estimating true value of {description[:20]}...")
    return {"description": description, "estimated_true_value": 300}

In [ ]:
def notify_user_of_deal(description: str, deal_price: float, estimated_true_value: float):
    print(f"Notifying user of {description} which costs {deal_price} and estimate is {estimated_true_value}")
    return {"notification_sent": "ok"}

## Telling the LLM about the tools it can use, with JSON

"Tool calling" is giving an LLM the power to run code on your computer!

Sounds a bit spooky?

The way it works is a little more mundane. We give the LLM a description of each Tool and the parameters, and we let it inform us if it wants any tool to be run.

It's not like OpenAI reaches in and runs a function. In the end, we have an if statement that calls our function if the model requests it.

In [ ]:
scan_function = {
    "name": "scan_the_internet_for_bargains",
    "description": "Returns top bargains scraped from the internet along with the price each item is being offered for",
    "parameters": {
        "type": "object",
        "properties": {},
        "required": [],
        "additionalProperties": False
    }
}

In [ ]:
estimate_function = {
    "name": "estimate_true_value",
    "description": "Given the description of an item, estimate how much it is actually worth",
    "parameters": {
        "type": "object",
        "properties": {
            "description": {
                "type": "string",
                "description": "The description of the item to be estimated"
            },
        },
        "required": ["description"],
        "additionalProperties": False
    }
}

In [ ]:
notify_function = {
    "name": "notify_user_of_deal",
    "description": "Send the user a push notification about the most compelling deal",
    "parameters": {
        "type": "object",
        "properties": {
            "description": {
                "type": "string",
                "description": "The description of the item"
            },
            "deal_price": {
                "type": "number",
                "description": "The price offered by this deal scraped from the internet"
            }
            ,
            "estimated_true_value": {
                "type": "number",
                "description": "The estimated actual value that this is worth"
            }
        },
        "required": ["description", "deal_price", "estimated_true_value"],
        "additionalProperties": False
    }
}

In [ ]:
tools = [{"type": "function", "function": scan_function},
        {"type": "function", "function": estimate_function},
        {"type": "function", "function": notify_function}]

## And now to bring it together - Tool calling in action

In [ ]:
mapping = {"scan_the_internet_for_bargains": scan_the_internet_for_bargains, "estimate_true_value": estimate_true_value, "notify_user_of_deal": notify_user_of_deal}

def handle_tool_call(message):
    results = []
    for tool_call in message.tool_calls:
        tool_name = tool_call.function.name
        arguments = json.loads(tool_call.function.arguments)
        print(f"Tool called: {tool_name}", flush=True)
        tool = mapping.get(tool_name)
        result = tool(**arguments) if tool else {}
        results.append({"role": "tool","content": json.dumps(result),"tool_call_id": tool_call.id})
    return results

In [ ]:
system_message = "You are an Autonomous AI Agent that makes use of tools to carry out your mission. Your mission is to find great deals on bargain products, and notify the user when you find them."
user_message = "Your mission is to discover great deals on products. First you should scan the internet for bargain deals. Then for each deal, you should estimate its true value - how much it's actually worth. "
user_message += "Finally, you should pick the single most compelling deal where the deal price is much lower than the estimated true value, and send the user a push notification about that deal. "
user_message += "You must only notify the user about one deal, and be sure to pick the most compelling deal, where the deal price is much lower than the estimated true value. Then just respond OK to indicate success."
messages = [{"role": "system", "content": system_message},{"role": "user", "content": user_message}]

In [ ]:
# A loop that repeatedly calls gpt-4o-mini until it has no more tools to call

done = False
while not done:
    response = openai.chat.completions.create(model=MODEL, messages=messages, tools=tools)
    if response.choices[0].finish_reason=="tool_calls":
        message = response.choices[0].message
        results = handle_tool_call(message)
        messages.append(message)
        messages.extend(results)
    else:
        done = True

print(response.choices[0].message.content)

## And now - putting all of this into a Planning Agent

In [ ]:
from agents.autonomous_planning_agent import AutonomousPlanningAgent

In [ ]:
import logging
root = logging.getLogger()
root.setLevel(logging.INFO)

In [ ]:
import chromadb
DB = "products_vectorstore"
client = chromadb.PersistentClient(path=DB)
collection = client.get_or_create_collection('products')

In [ ]:
agent = AutonomousPlanningAgent(collection)

In [ ]:
result = agent.plan()

# Finally - with a Gradio UI

In [ ]:
# Reset memory back to 2 deals discovered in the past

from deal_agent_framework import DealAgentFramework
DealAgentFramework.reset_memory()

In [ ]:
!python price_is_right.py